In [2]:
# !pip install llama-index-utils-workflow
# !pip install llama-index-llms-huggingface-api llama-index-embeddings-huggingface
# !pip install python-dotenv
# !pip install llama-index-llms-groq
# !pip install llama-index-vector-stores-chroma
# !pip install llama-index-tools-google

## Simple setup

In [3]:
from llama_index.llms.groq import Groq
import os
from google.colab import userdata

llm = Groq(model="llama3-70b-8192", api_key=userdata.get('groq_api'))

response = llm.complete("Hello, how are you?")
print(response)


I'm just a language model, I don't have emotions or feelings like humans do, so I don't have good or bad days. However, I'm functioning properly and ready to assist you with any questions or tasks you may have! How can I help you today?


## Basic Workflow Creation


In [4]:
from llama_index.core.workflow import StartEvent, StopEvent, Workflow, step

class MyWorkflow(Workflow):
  @step
  async def my_step(self, ev: StartEvent)->StopEvent:
    return StopEvent(result="Hello, world!")


w = MyWorkflow(timeout=10, verbose=False)
result = await w.run()

In [5]:
result

'Hello, world!'

## Connecting Multiple Steps


To connect multiple steps, we create custom events that carry data between steps. To do so, we need to add an Event that is passed between the steps and transfers the output of the first step to the second step.



In [6]:
from llama_index.core.workflow import Event

class ProcessingEvent(Event):
    intermediate_result: str

class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent) -> ProcessingEvent:
        # Process initial data
        return ProcessingEvent(intermediate_result="Step 1 complete")

    @step
    async def step_two(self, ev: ProcessingEvent) -> StopEvent:
        # Use the intermediate result
        final_result = f"Finished processing: {ev.intermediate_result}"
        return StopEvent(result=final_result)

w = MultiStepWorkflow(timeout=10, verbose=False)
result = await w.run()
result

'Finished processing: Step 1 complete'

In [ ]:
from llama_index.core.workflow import Event

class ProcessingEvent(Event):
    intermediate_result: str

class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent) -> ProcessingEvent:
        # Process initial data
        return ProcessingEvent(intermediate_result="Step 1 complete")

    @step
    async def step_two(self, ev: ProcessingEvent) -> StopEvent:
        # Use the intermediate result
        final_result = f"Finished processing: {ev.intermediate_result}"
        return StopEvent(result=final_result)

w = MultiStepWorkflow(timeout=10, verbose=False)
result = await w.run()
result

## Loops and Branches

The type hinting is the most powerful part of workflows because it allows us to create branches, loops, and joins to facilitate more complex workflows.




---

### Visual Summary of Flow:
1. `step_one(StartEvent)` → `LoopEvent` → rerun `step_one`
2. `step_one(LoopEvent)` → maybe `LoopEvent` again → rerun
3. Eventually `step_one(...)` → `ProcessingEvent` → triggers `step_two`

In [15]:
from llama_index.core.workflow import Event
import random

class ProcessingEvent(Event):
  intermediate_result: str

class LoopEvent(Event):
  loop_output: str

class MultiStepWorkflow(Workflow):
  @step
  async def step_one(self, ev: StartEvent | LoopEvent) -> ProcessingEvent | LoopEvent:
    if random.randint(0,1)==0:
      """
      So when step_one returns a LoopEvent, the workflow framework routes the returned
      event back to the appropriate handler, which in this case is again step_one,
      because step_one accepts LoopEvent as input.

      This essentially creates a loop until a ProcessingEvent is returned, which then
      gets routed to step_two.


      """
      print("bad thing happed")
      return LoopEvent(loop_output="back to step one.")

    else:
      print("good thing happened")
      return ProcessingEvent(intermediate_result="Step 1 complete")

  @step
  async def step_two(self, ev: ProcessingEvent) -> StopEvent:
    final_result = f"Finished processing: {ev.intermediate_result}"
    return StopEvent(result=final_result)

In [16]:
w = MultiStepWorkflow(verbose=False)
result = await w.run()
result

bad thing happed
bad thing happed
bad thing happed
bad thing happed
bad thing happed
bad thing happed
good thing happened


'Finished processing: Step 1 complete'

## Drawing Workflows

In [17]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(w, "flow.html")

flow.html


## State Management


State management is useful when you want to keep track of the state of the workflow, so that every step has access to the same state. We can do this by using the Context type hint on top of a parameter in the step function.



In [ ]:
from llama_index.core.workflow import Context, StartEvent, StopEvent


@step
async def query(self, ctx: Context, ev: StartEvent) -> StopEvent:
    # store query in the context
    await ctx.set("query", "What is the capital of France?")

    # do something with context and event
    val = ...

    # retrieve query from the context
    query = await ctx.get("query")

    return StopEvent(result=val)

## Automating workflows with Multi-Agent Workflows


In [18]:
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent

# Define some tools
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [19]:
llm

Groq(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7e1e99647910>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7e1f79ebfce0>, completion_to_prompt=<function default_completion_to_prompt at 0x7e1f79a1de40>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='llama3-70b-8192', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='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', api_base='https://api.groq.com/openai/v1', api_version='', strict=False, reasoning_effort=None, modalities=None, audio_config=None, context_window=3900, is_chat_model=True, is_function_calling_model=True, tokenizer=None)

we can pass functions directly without FunctionTool -- the fn/docstring are parsed for the name/description


In [20]:
multiply_agent= ReActAgent(
    name="multiply_agent",
    description="Is able to multiply two integers",
    system_prompt="A helpful assistant that can use a tool to multiply numbers.",
    tools=[multiply],
    llm=llm,
)

In [21]:
addition_agent = ReActAgent(
    name="add_agent",
    description="Is able to add two integers",
    system_prompt="A helpful assistant that can use a tool to add numbers.",
    tools=[add],
    llm=llm,
)

In [22]:
# Create the workflow
workflow = AgentWorkflow(
    agents=[multiply_agent, addition_agent],
    root_agent="multiply_agent",
)

In [23]:
response = await workflow.run(user_msg="Can you add 5 and 3?")


In [24]:
print(response)

8


In [25]:
await workflow.run(user_msg="Can you add 5 and 3 and then multipy by 4*5?")


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The result of adding 5 and 3 and then multiplying by 4*5 is 160.')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'to_agent': 'add_agent', 'reason': 'need to add 5 and 3'}, tool_id='b5f4fb65-4cb0-4956-b007-69c97c0e98fe', tool_output=ToolOutput(content='Agent add_agent is now handling the request due to the following reason: need to add 5 and 3.\nPlease continue with the current request.', tool_name='handoff', raw_input={'args': (), 'kwargs': {'to_agent': 'add_agent', 'reason': 'need to add 5 and 3'}}, raw_output='Agent add_agent is now handling the request due to the following reason: need to add 5 and 3.\nPlease continue with the current request.', is_error=False), return_direct=True), ToolCallResult(tool_name='add', tool_kwargs={'a': 5, 'b': 3}, tool_id='7848267d-c637-4036-8190-61e2c0dc09f5', tool_output=ToolOutput(content='8', 

In [26]:
response=await workflow.run(user_msg="Can you add 5 and 3 and then multipy by 4*5?")


In [28]:
print(response)

The result of adding 5 and 3 and then multiplying by 4*5 is 160.


In [30]:
from llama_index.core.workflow import Context

# Define some tools
async def add(ctx: Context, a: int, b: int) -> int:
    """Add two numbers."""
    # update our count
    cur_state = await ctx.get("state")
    cur_state["num_fn_calls"] += 1
    await ctx.set("state", cur_state)

    return a + b

async def multiply(ctx: Context, a: int, b: int) -> int:
    """Multiply two numbers."""
    # update our count
    cur_state = await ctx.get("state")
    cur_state["num_fn_calls"] += 1
    await ctx.set("state", cur_state)

    return a * b

...

workflow = AgentWorkflow(
    agents=[multiply_agent, addition_agent],
    root_agent="multiply_agent",
    initial_state={"num_fn_calls": 0},
    state_prompt="Current state: {state}. User message: {msg}",
)

# run the workflow with context
ctx = Context(workflow)
response = await workflow.run(user_msg="Can you add 5 and 3?", ctx=ctx)

# pull out and inspect the state
state = await ctx.get("state")
print(state["num_fn_calls"])

0
